# Bayesian Optimization
https://github.com/fmfn/BayesianOptimization

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
import numpy as np
from matplotlib import pyplot as mp
%matplotlib inline
np.random.seed(0)

# 基本用法
https://github.com/fmfn/BayesianOptimization/blob/master/examples/visualization.ipynb

#### 安装
pip install bayesian-optimization

In [2]:
from bayes_opt import BayesianOptimization

#### 目标函数
$ f(x, y) = 1 - x^2 - (y - 1)^2 $

In [3]:
f = lambda x, y: 1 - x ** 2 - (y - 1) ** 2

#### 参数范围

In [4]:
pbounds = {'x': (-1, 4), 'y': (-3, 3)}  # Bounded region of parameter space

#### 创建优化器

In [5]:
optimizer = BayesianOptimization(f=f, pbounds=pbounds)

#### 优化

In [6]:
optimizer.maximize(init_points=3, n_iter=7)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
|  1        | -8.883    | -0.9299   | -2.003    |
|  2        |  0.7738   |  0.1734   |  1.443    |
|  3        | -0.05796  |  0.5309   |  1.881    |
|  4        |  0.4119   | -0.2153   |  1.736    |
|  5        |  0.9339   | -0.2568   |  0.9899   |
|  6        |  0.6482   |  0.4986   |  0.6788   |
|  7        | -2.19     |  1.779    |  0.841    |
|  8        |  0.6725   | -0.05958  |  0.4308   |
|  9        | -0.004328 | -1.0      |  1.066    |
|  10       | -31.0     |  4.0      | -3.0      |


#### 结果

In [8]:
optimizer.res

[{'target': -8.883353398300128,
  'params': {'x': -0.9298568422167244, 'y': -2.0031183212259336}},
 {'target': 0.773827821237246,
  'params': {'x': 0.1733803324412544, 'y': 1.4428447121568846}},
 {'target': -0.057955383613888856,
  'params': {'x': 0.5309167680987894, 'y': 1.8809555998831184}},
 {'target': 0.41190288251496376,
  'params': {'x': -0.21534651286065887, 'y': 1.7360183400458105}},
 {'target': 0.9339288459782804,
  'params': {'x': -0.25684360952250557, 'y': 0.989875066954772}},
 {'target': 0.6482354744434342,
  'params': {'x': 0.49860226972980015, 'y': 0.6788142248217436}},
 {'target': -2.1900750759403977,
  'params': {'x': 1.7789903370425315, 'y': 0.8410394493916877}},
 {'target': 0.6724500805542406,
  'params': {'x': -0.05958475232290208, 'y': 0.4307903929690067}},
 {'target': -0.0043281566081576975,
  'params': {'x': -1.0, 'y': 1.0657887270598672}},
 {'target': -31.0, 'params': {'x': 4.0, 'y': -3.0}}]

In [7]:
optimizer.max

{'target': 0.9339288459782804,
 'params': {'x': -0.25684360952250557, 'y': 0.989875066954772}}

In [9]:
optimizer.space.target

array([-8.88335340e+00,  7.73827821e-01, -5.79553836e-02,  4.11902883e-01,
        9.33928846e-01,  6.48235474e-01, -2.19007508e+00,  6.72450081e-01,
       -4.32815661e-03, -3.10000000e+01])

In [10]:
optimizer.space.target.argmax()

4

# 结合sklearn
https://github.com/fmfn/BayesianOptimization/blob/master/examples/sklearn_example.py

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')  # 不打印警告
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from bayes_opt import BayesianOptimization


ls_of_clf = []


def load_data():
    X, y = make_circles(n_samples=3000, noise=.3, factor=.4)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return X_train, y_train, X_test, y_test


def score(X, y, C, gamma):
    clf = SVC(C=C, gamma=gamma).fit(X, y)
    ls_of_clf.append(clf)
    return clf.score(X, y)


def best_clf(X, y):
    f = lambda C, gamma: score(X, y, C, gamma)
    pbounds = {'C': (1e-2, 1e1), 'gamma': (1e-4, 1e-1)}
    optimizer = BayesianOptimization(f, pbounds)
    optimizer.maximize(n_iter=10)
    print('Best result:', optimizer.max)
    idx = optimizer.space.target.argmax()
    return ls_of_clf[idx], optimizer.max['params']


X_train, y_train, X_test, y_test = load_data()
clf, params = best_clf(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print(SVC(**params).fit(X_train, y_train).score(X_train, y_train))
print(SVC(**params).fit(X_train, y_train).score(X_test, y_test))
print(SVC().fit(X_train, y_train).score(X_train, y_train))
print(SVC().fit(X_train, y_train).score(X_test, y_test))

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.8053   |  8.984    |  0.03623  |
|  2        |  0.8093   |  3.71     |  0.0846   |
|  3        |  0.7924   |  7.566    |  0.01803  |
|  4        |  0.6582   |  1.453    |  0.01816  |
|  5        |  0.7933   |  2.716    |  0.02867  |
|  6        |  0.5036   |  10.0     |  0.0001   |
|  7        |  0.5036   |  0.01     |  0.1      |
|  8        |  0.5036   |  5.46     |  0.0001   |
|  9        |  0.808    |  8.278    |  0.1      |
|  10       |  0.5036   |  8.402    |  0.0001   |
|  11       |  0.808    |  6.522    |  0.1      |
|  12       |  0.8067   |  4.539    |  0.1      |
|  13       |  0.5036   |  0.7376   |  0.0001   |
|  14       |  0.8071   |  2.12     |  0.1      |
|  15       |  0.8089   |  9.437    |  0.1      |
Best result: {'target': 0.8093333333333333, 'params': {'C': 3.709584138875576, 'gamma': 0.08459699681061882}}
0.8093333333333333
0.8186666666666667
0.